# TODO

# Set-up

In [1]:
import os
import sys
import random
import time
import pandas as pd
import numpy as np

# ignore pandas warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
os.chdir("/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/aracne/bin/igvf_b01_LeftCortex")
os.getcwd()

'/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/aracne/bin/igvf_b01_LeftCortex'

# Load file metadata

In [3]:
# Where the input metadata is stored for all files to run on
in_dir = "/cellar/users/aklie/data/igvf/topic_grn_links/subsets/for_regulon_inference/igvf_b01_LeftCortex/31Aug23/3744376661_20230829-175127"

In [4]:
# Load the file metadata
file_metadata = pd.read_csv(os.path.join(in_dir, "file_metadata.tsv"), sep="\t")
file_metadata.head()

,file,file_type,cells,genes,normalization,n_cells,n_genes,percent_int,percent_0,percent_less_than_0
0,random_500.random_2000.raw.h5ad,.h5ad,random_500,random_2000,raw,500,2000,1.000000,0.937368,0.0
1,random_500.random_2000.raw.loom,.loom,random_500,random_2000,raw,500,2000,1.000000,0.937368,0.0
2,random_500.random_2000.raw.tsv,.tsv,random_500,random_2000,raw,2000,500,1.000000,0.937368,0.0
3,random_500.random_2000.log1p_cp10k.h5ad,.h5ad,random_500,random_2000,log1p_cp10k,500,2000,0.937368,0.937368,0.0
4,random_500.random_2000.log1p_cp10k.loom,.loom,random_500,random_2000,log1p_cp10k,500,2000,0.937368,0.937368,0.0


# Extract runs of interest

In [5]:
# Parameters for runs of interest
genes = ["0.05"] # could be ["0.01", "0.05", "0.1", "0.2"]
cells = ["balanced_genotype_microglia"]
normalizations = ["raw", "log1p_cp10k", "pf_log1p_pf", "cpm_rank"]
file_types = [".tsv"]  # could be [".h5ad", ".loom", ".tsv"], but selecting tsv for ARACNe
out_dir = "/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/aracne/results/igvf_b01_LeftCortex"

In [6]:
# Set up masks for the file metadata based on above
genes_mask = file_metadata["genes"].isin(genes)
cells_mask = file_metadata["cells"].isin(cells)
normalizations_mask = file_metadata["normalization"].isin(normalizations)
file_types_mask = file_metadata["file_type"].isin(file_types)

In [7]:
# Extract the files to run
curr_metadata = file_metadata[genes_mask & cells_mask & normalizations_mask & file_types_mask]
curr_metadata["in_file"] = curr_metadata["cells"] + "/" + curr_metadata["genes"].astype(str) + "/" + curr_metadata["normalization"] + "/" + curr_metadata["file"]
curr_metadata["in_file_path"] = curr_metadata["in_file"].apply(lambda x: os.path.join(in_dir, x))
curr_metadata["genotype"] = [i.split(".")[0].split('-')[-1] for i in curr_metadata["file"]] 
curr_metadata["out_dir"] = curr_metadata["cells"] + "/" + curr_metadata["genes"].astype(str) + "/" + curr_metadata["normalization"] + "/" + curr_metadata["genotype"]
curr_metadata["out_dir_path"] = curr_metadata["out_dir"].apply(lambda x: os.path.join(out_dir, x))
curr_metadata

,file,file_type,cells,genes,normalization,n_cells,n_genes,percent_int,percent_0,percent_less_than_0,in_file,in_file_path,genotype,out_dir,out_dir_path
486,balanced_genotype_microglia-B6J.0.05.raw.tsv,.tsv,balanced_genotype_microglia,0.05,raw,4077,1206,1.000000,0.840221,0.000000,balanced_genotype_microglia/0.05/raw/balanced_...,/cellar/users/aklie/data/igvf/topic_grn_links/...,B6J,balanced_genotype_microglia/0.05/raw/B6J,/cellar/users/aklie/projects/igvf/topic_grn_li...
487,balanced_genotype_microglia-CASTJ.0.05.raw.tsv,.tsv,balanced_genotype_microglia,0.05,raw,4077,1206,1.000000,0.842789,0.000000,balanced_genotype_microglia/0.05/raw/balanced_...,/cellar/users/aklie/data/igvf/topic_grn_links/...,CASTJ,balanced_genotype_microglia/0.05/raw/CASTJ,/cellar/users/aklie/projects/igvf/topic_grn_li...
488,balanced_genotype_microglia-both.0.05.raw.tsv,.tsv,balanced_genotype_microglia,0.05,raw,4077,2412,1.000000,0.841505,0.000000,balanced_genotype_microglia/0.05/raw/balanced_...,/cellar/users/aklie/data/igvf/topic_grn_links/...,both,balanced_genotype_microglia/0.05/raw/both,/cellar/users/aklie/projects/igvf/topic_grn_li...
495,balanced_genotype_microglia-B6J.0.05.log1p_cp1...,.tsv,balanced_genotype_microglia,0.05,log1p_cp10k,4077,1206,0.840221,0.840221,0.000000,balanced_genotype_microglia/0.05/log1p_cp10k/b...,/cellar/users/aklie/data/igvf/topic_grn_links/...,B6J,balanced_genotype_microglia/0.05/log1p_cp10k/B6J,/cellar/users/aklie/projects/igvf/topic_grn_li...
496,balanced_genotype_microglia-CASTJ.0.05.log1p_c...,.tsv,balanced_genotype_microglia,0.05,log1p_cp10k,4077,1206,0.842789,0.842789,0.000000,balanced_genotype_microglia/0.05/log1p_cp10k/b...,/cellar/users/aklie/data/igvf/topic_grn_links/...,CASTJ,balanced_genotype_microglia/0.05/log1p_cp10k/C...,/cellar/users/aklie/projects/igvf/topic_grn_li...
497,balanced_genotype_microglia-both.0.05.log1p_cp...,.tsv,balanced_genotype_microglia,0.05,log1p_cp10k,4077,2412,0.841505,0.841505,0.000000,balanced_genotype_microglia/0.05/log1p_cp10k/b...,/cellar/users/aklie/data/igvf/topic_grn_links/...,both,balanced_genotype_microglia/0.05/log1p_cp10k/both,/cellar/users/aklie/projects/igvf/topic_grn_li...
504,balanced_genotype_microglia-B6J.0.05.pf_log1p_...,.tsv,balanced_genotype_microglia,0.05,pf_log1p_pf,4077,1206,0.840221,0.840221,0.000000,balanced_genotype_microglia/0.05/pf_log1p_pf/b...,/cellar/users/aklie/data/igvf/topic_grn_links/...,B6J,balanced_genotype_microglia/0.05/pf_log1p_pf/B6J,/cellar/users/aklie/projects/igvf/topic_grn_li...
505,balanced_genotype_microglia-CASTJ.0.05.pf_log1...,.tsv,balanced_genotype_microglia,0.05,pf_log1p_pf,4077,1206,0.842789,0.842789,0.000000,balanced_genotype_microglia/0.05/pf_log1p_pf/b...,/cellar/users/aklie/data/igvf/topic_grn_links/...,CASTJ,balanced_genotype_microglia/0.05/pf_log1p_pf/C...,/cellar/users/aklie/projects/igvf/topic_grn_li...
506,balanced_genotype_microglia-both.0.05.pf_log1p...,.tsv,balanced_genotype_microglia,0.05,pf_log1p_pf,4077,2412,0.841505,0.841505,0.000000,balanced_genotype_microglia/0.05/pf_log1p_pf/b...,/cellar/users/aklie/data/igvf/topic_grn_links/...,both,balanced_genotype_microglia/0.05/pf_log1p_pf/both,/cellar/users/aklie/projects/igvf/topic_grn_li...
513,balanced_genotype_microglia-B6J.0.05.cpm_rank.tsv,.tsv,balanced_genotype_microglia,0.05,cpm_rank,4077,1206,0.013455,0.002903,0.498555,balanced_genotype_microglia/0.05/cpm_rank/bala...,/cellar/users/aklie/data/igvf/topic_grn_links/...,B6J,balanced_genotype_microglia/0.05/cpm_rank/B6J,/cellar/users/aklie/projects/igvf/topic_grn_li...


In [8]:
time_id = time.strftime("%Y%m%d-%H%M%S")
run_id =  random.getrandbits(32)
run = str(run_id) + "_" + time_id

In [9]:
curr_run = curr_metadata[["in_file_path", "out_dir_path"]]
curr_run["tf_list"] = "/cellar/users/aklie/opt/igvf-ucsd/aracne_pipeline/data/tf_cotf_signalling_list/mouse/tf_mus_symbol.txt"
curr_run["pval"] = "1e-8"
curr_run["num_boostraps"] = 5
curr_run["tf_type"] = "tf"
curr_run["out_dir_path"] = curr_run["out_dir_path"] +  "/" + curr_run["tf_type"]
curr_run["run_id"] = run
curr_run.to_csv(f"{run}_paths.csv", index=False)

# DONE

---